In [ ]:
https://moez-62905.medium.com/optimize-sales-funnel-and-increase-revenue-with-machine-learning-in-python-9af57c1e5776

In [1]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 2.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as GBC, RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, auc, roc_curve
import category_encoders as ce
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/classification/Lead Prediction/lead.csv')
df

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,...,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Lead Profile,City,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,No,No,0,0.0,0,0.00,Page Visited on Website,missing,...,No,No,No,No,No,No,Select,Select,No,Modified
1,API,Organic Search,No,No,0,5.0,674,2.50,Email Opened,India,...,No,No,No,No,No,No,Select,Select,No,Email Opened
2,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,Email Opened,India,...,No,No,No,No,No,No,Potential Lead,Mumbai,Yes,Email Opened
3,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,Unreachable,India,...,No,No,No,No,No,No,Select,Mumbai,No,Modified
4,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,Converted to Lead,India,...,No,No,No,No,No,No,Select,Mumbai,No,Modified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9098,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,Email Marked Spam,Saudi Arabia,...,No,No,No,No,No,No,Potential Lead,Mumbai,No,Email Marked Spam
9099,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,SMS Sent,India,...,No,No,No,No,No,No,Potential Lead,Mumbai,Yes,SMS Sent
9100,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,SMS Sent,India,...,No,No,No,No,No,No,Potential Lead,Mumbai,Yes,SMS Sent
9101,Landing Page Submission,Google,No,No,1,3.0,499,3.00,SMS Sent,India,...,No,No,No,No,No,No,missing,Other Metro Cities,No,SMS Sent


In [4]:
out_column = 'Converted'
to_remove = []
features = [c for c in df.columns if (c not in [out_column]) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() == 2)]
numerical = [col for col in features if col not in categorical + binary]
df[numerical] = df[numerical].apply(pd.to_numeric,1)

#df.replace({out_column: {'no': 0, 'yes': 1}}, inplace=True)
fig = go.Figure(data=go.Heatmap(z=df[numerical].corr(),x=numerical,y=numerical))
fig.show()

In [5]:
for col in categorical:
    counts = df[col].value_counts()
    df = df[~df[col].isin(counts[counts < 4].index)]
df = df.reset_index(drop=True)
df

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,...,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Lead Profile,City,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,No,No,0,0.0,0.0,0.0,Page Visited on Website,missing,...,No,No,No,No,No,No,Select,Select,No,Modified
1,API,Organic Search,No,No,0,5.0,674.0,2.5,Email Opened,India,...,No,No,No,No,No,No,Select,Select,No,Email Opened
2,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532.0,2.0,Email Opened,India,...,No,No,No,No,No,No,Potential Lead,Mumbai,Yes,Email Opened
3,Landing Page Submission,Direct Traffic,No,No,0,1.0,305.0,1.0,Unreachable,India,...,No,No,No,No,No,No,Select,Mumbai,No,Modified
4,Landing Page Submission,Google,No,No,1,2.0,1428.0,1.0,Converted to Lead,India,...,No,No,No,No,No,No,Select,Mumbai,No,Modified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9040,API,Direct Traffic,No,No,1,13.0,1409.0,2.6,SMS Sent,India,...,No,No,No,No,No,No,Select,Select,No,SMS Sent
9041,Landing Page Submission,Direct Traffic,No,No,1,5.0,210.0,2.5,SMS Sent,India,...,No,No,No,No,No,No,Potential Lead,Mumbai,No,Modified
9042,Landing Page Submission,Direct Traffic,No,No,0,2.0,238.0,2.0,SMS Sent,India,...,No,No,No,No,No,No,Potential Lead,Mumbai,Yes,SMS Sent
9043,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199.0,2.0,SMS Sent,India,...,No,No,No,No,No,No,Potential Lead,Mumbai,Yes,SMS Sent


In [6]:
from sklearn.model_selection import train_test_split

OE = OneHotEncoder(drop='if_binary', sparse=False)
df[binary] = OE.fit_transform(df[binary]).astype(int)

features = categorical + numerical + binary
X_train, X_test, Y_train, Y_test = train_test_split(df[features], df[out_column], test_size=0.15, random_state=0, stratify=df[out_column])

encoder = ce.CatBoostEncoder(verbose=1, cols=categorical)
encoder.fit(X_train[categorical], Y_train)
X_train[categorical] = encoder.transform(X_train[categorical])
X_test[categorical] = encoder.transform(X_test[categorical])
 
X_train.shape, X_test.shape

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



((7688, 23), (1357, 23))

In [7]:
hp = [6.50000000e+01, 1.37458499e-01, 3.00000000e+00, 9.21149918e-04,
       2.57279157e-01]
hp = [np.round(hp[0]).astype(int), hp[1], np.round(hp[2]).astype(int), hp[3], hp[4]]
clf = GBC(n_estimators=hp[0], min_samples_split=hp[1], max_features=hp[2], min_samples_leaf=hp[3],
               learning_rate=hp[4], random_state=1)
#regr = GBR(min_samples_leaf=6, max_depth=20, random_state=10)
clf.fit(X_train, Y_train)

GradientBoostingClassifier(learning_rate=0.257279157, max_features=3,
                           min_samples_leaf=0.000921149918,
                           min_samples_split=0.137458499, n_estimators=65,
                           random_state=1)

In [11]:
hp = [9.00000000e+01, 6.39835461e-03, 1.00000000e-05, 4.00000000e+00]
hp = [np.round(hp[0]).astype(int), hp[1], hp[2], np.round(hp[3]).astype(int)]
clf = RFC(n_estimators=hp[0], n_jobs=-1, min_samples_split=hp[1], min_samples_leaf=hp[2], max_features=hp[3],
               bootstrap=True, random_state=1)
clf.fit(X_train, Y_train)

RandomForestClassifier(max_features=4, min_samples_leaf=1e-05,
                       min_samples_split=0.00639835461, n_estimators=90,
                       n_jobs=-1, random_state=1)

In [12]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)
score = cross_val_score(clf, X_train, Y_train, cv=kf, scoring='f1').mean()
score

0.7842501056497927

In [13]:
clf.score(X_train, Y_train), clf.score(X_test, Y_test)

(0.8677159209157128, 0.8555637435519529)

In [14]:
pred = clf.predict_proba(X_train)[:,1]
auc = roc_auc_score(Y_train, pred)
pd.DataFrame({'AUC': auc, 'ACC': accuracy_score(Y_train, np.round(pred)), 'PRE': precision_score(Y_train, np.round(pred)), 'REC': recall_score(Y_train, np.round(pred)), 'F1':f1_score(Y_train, np.round(pred))}, index=[0])

,AUC,ACC,PRE,REC,F1
0,0.941996,0.867716,0.848795,0.793987,0.820477


In [15]:
pred = clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(Y_test, pred)
pd.DataFrame({'AUC': auc, 'ACC': accuracy_score(Y_test, np.round(pred)), 'PRE': precision_score(Y_test, np.round(pred)), 'REC': recall_score(Y_test, np.round(pred)), 'F1':f1_score(Y_test, np.round(pred))}, index=[0])

,AUC,ACC,PRE,REC,F1
0,0.928714,0.855564,0.829569,0.781431,0.804781


In [16]:
confusion_matrix(Y_test, np.round(pred))

array([[757,  83],
       [113, 404]])

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import fbeta_score, make_scorer


result = permutation_importance(clf, X_test, Y_test, n_repeats=10, random_state=42, n_jobs=2, scoring='f1')
fig = go.Figure()
# Use x instead of y argument for horizontal plot
for i in range(X_train.shape[1]):
    fig.add_trace(go.Box(x=result['importances'][i], name=X_test.columns[i]))
fig.show()